## Assignment Exercises

**Note:** All the exercises will be based off the current date of Saturday 4/25/2020. Monday is considered to be the start of each week.

Details on the dataset used can be found in the section Sample Data.

### Part A

1) Write a query that we can use to create a plot of the total count of rides completed in our San Francisco region, in each week over the last 12 weeks.

**Query:**
```
SELECT 
DATE(DATE_TRUNC('week', start_time)) AS week_start,
COUNT(ride_id) AS ride_count
FROM rides 
WHERE ride_region = 'San Francisco'
AND is_completed = 'Y'
GROUP BY week_start
ORDER BY week_start desc
LIMIT 12;
```

In [117]:
cur.execute("""SELECT 
                DATE(DATE_TRUNC('week', start_time)) AS week_start,
                COUNT(ride_id) AS ride_count
                FROM rides 
                WHERE ride_region = 'San Francisco'
                AND is_completed = 'Y'
                GROUP BY week_start
                ORDER BY week_start desc
                LIMIT 12;
                """)
rows = cur.fetchall()
conn.commit()

print(f"Number of rows: {len(rows)}")
print ("\nShow me the databases:\n")
for row in rows:
    print("   ", row)

Number of rows: 12

Show me the databases:

    (datetime.date(2020, 4, 20), 6)
    (datetime.date(2020, 4, 13), 4)
    (datetime.date(2020, 4, 6), 4)
    (datetime.date(2020, 3, 30), 3)
    (datetime.date(2020, 3, 23), 5)
    (datetime.date(2020, 3, 16), 6)
    (datetime.date(2020, 3, 9), 5)
    (datetime.date(2020, 3, 2), 8)
    (datetime.date(2020, 2, 24), 7)
    (datetime.date(2020, 2, 17), 4)
    (datetime.date(2020, 2, 10), 8)
    (datetime.date(2020, 2, 3), 3)


**Verification**

In [118]:
import warnings
warnings.filterwarnings("ignore")

sf_rides = rides[(rides.ride_region == 'San Francisco') & (rides.is_completed == 'Y')]
sf_rides['week_start'] = sf_rides['start_time'].dt.to_period('W').apply(lambda r: r.start_time)
sf_rides.groupby('week_start').count()['ride_id'].reset_index().sort_values('week_start', ascending = False)[:12]

,week_start,ride_id
18,2020-04-20,6
17,2020-04-13,4
16,2020-04-06,4
15,2020-03-30,3
14,2020-03-23,5
13,2020-03-16,6
12,2020-03-09,5
11,2020-03-02,8
10,2020-02-24,7
9,2020-02-17,4


2) Write a query that returns a count of the total number of weekly drivers based in San Francisco to complete at least one ride in each week over the last 12 weeks.

**Query:**
```
WITH d_weeks AS (
    SELECT 
    rides.driver_id,
    COUNT(DISTINCT DATE(DATE_TRUNC('week', start_time))) AS weeks_worked
    FROM rides, drivers
    WHERE rides.driver_id = drivers.driver_id
    AND home_region = 'San Francisco'
    AND is_completed = 'Y'
    AND DATE(DATE_TRUNC('week', start_time)) > (DATE_TRUNC('week', current_date) - interval '12 weeks')
    GROUP BY rides.driver_id)

SELECT
COUNT(*) FILTER (WHERE weeks_worked = 12)
FROM d_weeks
;
```

In [142]:
cur.execute("""
WITH d_weeks AS (
    SELECT 
    rides.driver_id,
    COUNT(DISTINCT DATE(DATE_TRUNC('week', start_time))) AS weeks_worked
    FROM rides, drivers
    WHERE rides.driver_id = drivers.driver_id
    AND home_region = 'San Francisco'
    AND is_completed = 'Y'
    AND DATE(DATE_TRUNC('week', start_time)) > (DATE_TRUNC('week', current_date) - interval '12 weeks')
    GROUP BY rides.driver_id)

SELECT
COUNT(*) FILTER (WHERE weeks_worked = 12)
FROM d_weeks
;
                """)
rows = cur.fetchall()
conn.commit()

print ("\nShow me the databases:\n")
for row in rows:
    print("   ", row)


Show me the databases:

    (2,)


**Verification**

In [141]:
d_rides = rides.merge(drivers[['driver_id', 'home_region']], how = 'left', on = 'driver_id')
d_rides['week_start'] = d_rides['start_time'].dt.to_period('W').apply(lambda r: r.start_time)

today = dt.date.today()
current_week = today - dt.timedelta(days=today.weekday())
current_week - dt.timedelta(weeks = 12)

d_rides = d_rides[(d_rides.is_completed == 'Y') & (d_rides.home_region == 'San Francisco') & (d_rides.week_start > (current_week - dt.timedelta(weeks = 12)))]
weekly_drivers = d_rides.groupby(['driver_id'])['week_start'].nunique().reset_index()
weekly_drivers[weekly_drivers.week_start == 12]

,driver_id,week_start
2,D27158,12
12,D96971,12


### Part B

We notice that ride counts fluctuate in a pattern depending on the day of the week. We want to remove the effect of these daily fluctuations so that we can see trends occurring on a week by week basis.

Write a query that returns, for each of the last 90 days, a count of the rides taken in the 7 day window preceding that day.

**Query:**
```
WITH dates AS(
SELECT rolling_7_date::date
FROM generate_series(current_date - interval '89 days', current_date, INTERVAL '1 day') rolling_7_date
)

SELECT
rolling_7_date,
COUNT(ride_id) AS ride_count
FROM dates
LEFT JOIN rides ON DATE(start_time) BETWEEN rolling_7_date - interval '7 days' AND rolling_7_date - interval '1 day'
GROUP BY rolling_7_date
ORDER BY rolling_7_date
;
```

In [146]:
cur.execute("""
WITH dates AS(
SELECT rolling_7_date::date
FROM generate_series(current_date - interval '89 days', current_date, INTERVAL '1 day') rolling_7_date
)

SELECT
rolling_7_date,
COUNT(ride_id) AS ride_count
FROM dates
LEFT JOIN rides ON DATE(start_time) BETWEEN rolling_7_date - interval '7 days' AND rolling_7_date - interval '1 day'
GROUP BY rolling_7_date
ORDER BY rolling_7_date
;
                """)
rows = cur.fetchall()
conn.commit()

print(f"Number of rows: {len(rows)}")
print ("\nShow me the databases:\n")
for row in rows:
    print("   ", row)

Number of rows: 90

Show me the databases:

    (datetime.date(2020, 1, 27), 12)
    (datetime.date(2020, 1, 28), 12)
    (datetime.date(2020, 1, 29), 14)
    (datetime.date(2020, 1, 30), 15)
    (datetime.date(2020, 1, 31), 16)
    (datetime.date(2020, 2, 1), 13)
    (datetime.date(2020, 2, 2), 14)
    (datetime.date(2020, 2, 3), 15)
    (datetime.date(2020, 2, 4), 14)
    (datetime.date(2020, 2, 5), 12)
    (datetime.date(2020, 2, 6), 11)
    (datetime.date(2020, 2, 7), 9)
    (datetime.date(2020, 2, 8), 10)
    (datetime.date(2020, 2, 9), 8)
    (datetime.date(2020, 2, 10), 8)
    (datetime.date(2020, 2, 11), 6)
    (datetime.date(2020, 2, 12), 10)
    (datetime.date(2020, 2, 13), 12)
    (datetime.date(2020, 2, 14), 12)
    (datetime.date(2020, 2, 15), 14)
    (datetime.date(2020, 2, 16), 15)
    (datetime.date(2020, 2, 17), 17)
    (datetime.date(2020, 2, 18), 21)
    (datetime.date(2020, 2, 19), 17)
    (datetime.date(2020, 2, 20), 16)
    (datetime.date(2020, 2, 21), 17)
    (da

## Sample Data

In [112]:
import pandas as pd
drivers = pd.read_csv('./Data/drivers.csv')
rides = pd.read_csv('./Data/rides.csv', parse_dates = ['start_time', 'end_time'])

### Drivers Dataset

The drivers dataset contains 50 unique drivers for 4 different regions:
* Alameda
* Daly City
* San Francisco
* Concord

In [148]:
drivers.head(5)

,driver_id,onboarding_time,home_region
0,D79396,11/22/2019 09:03 PM,Alameda
1,D83025,11/18/2019 10:46 PM,Daly City
2,D31884,12/12/2019 02:06 PM,Daly City
3,D72540,11/21/2019 01:52 PM,Alameda
4,D25768,11/12/2019 08:39 AM,San Francisco


In [110]:
print(drivers.describe())

       driver_id      onboarding_time    home_region
count         50                   50             50
unique        50                   50              4
top       D96971  12/27/2019 08:25 AM  San Francisco
freq           1                    1             16


### Rides Dataset

The rides dataset contains 220 unique rides during the time period of 12/21/2019 until 4/25/2020 for 4 different regions:
* Concord
* Daly City
* San Francisco
* Walnut Creek

There are 40 unique passengers on the app paired with drivers regardless of their indicated home region. Only 174 of the rides were completed.

In [147]:
rides.head(5)

,ride_id,start_time,end_time,passenger_id,driver_id,ride_region,is_completed
0,R80522,2020-02-12 17:59:00,2020-02-12 18:20:00,P51597,D60668,San Francisco,Y
1,R52714,2020-02-01 21:59:00,2020-02-01 22:28:00,P24165,D60668,San Francisco,Y
2,R12066,2020-03-24 19:36:00,2020-03-24 20:13:00,P61317,D64033,Walnut Creek,N
3,R23235,2019-12-22 20:28:00,2019-12-22 21:02:00,P51597,D48240,San Francisco,Y
4,R48034,2020-03-20 09:00:00,2020-03-20 09:51:00,P48745,D51051,Concord,Y


In [114]:
print(rides.describe())
print("")
print(f"Date range: {min(rides.start_time)} - {max(rides.start_time)}")

       ride_id           start_time             end_time passenger_id  \
count      220                  220                  220          220   
unique     220                  219                  220           40   
top     R97391  2020-02-11 21:31:00  2020-02-26 22:23:00       P22921   
freq         1                    2                    1           13   
first      NaN  2019-12-21 09:15:00  2019-12-21 10:14:00          NaN   
last       NaN  2020-04-25 20:52:00  2020-04-25 21:03:00          NaN   

       driver_id    ride_region is_completed  
count        220            220          220  
unique        50              4            2  
top       D96971  San Francisco            Y  
freq          13            109          174  
first        NaN            NaN          NaN  
last         NaN            NaN          NaN  

Date range: 2019-12-21 09:15:00 - 2020-04-25 20:52:00


## Load Data into Postgres Server

### Setting up Connection to Postgres Server

In [103]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=lyft user=postgres password=Postgres")
cur = conn.cursor()

### Create Table in Postgres Server

In [115]:
cur.execute("""CREATE TABLE drivers(
    driver_id TEXT PRIMARY KEY,
    onboarding_time TIMESTAMP,
    home_region TEXT
);

CREATE TABLE rides(
    ride_id TEXT PRIMARY KEY,
    start_time TIMESTAMP,
    end_time TIMESTAMP,
    passenger_id TEXT,
    driver_id TEXT,
    ride_region TEXT,
    is_completed TEXT
);
""")

conn.commit()

### Loading Data into Postgres Server 

In [116]:
with open('./Data/drivers.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'drivers', sep=',')

conn.commit()

with open('./Data/rides.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'rides', sep=',')

conn.commit()